In [0]:
def to_silver_calendar(path_bronze_calendar, path_silver_calendar):
    df_calendar = spark.read.parquet(path_bronze_calendar)

    df_calendar = (df_calendar
                   .withColumn("listing_id", col("listing_id").cast(LongType()))
                   .withColumn("date", to_date("date", "yyyy-MM-dd"))
                   .withColumn("available", 
                               when(col("available") == "t", True).
                               when(col("available") == "f", False).
                               otherwise(None))
                   .withColumn("price", regexp_replace(col("price"), "\\$", ""))
                   .withColumn("price", regexp_replace(col("price"), ",", ""))
                   .withColumn("price", col("price").cast("float"))
                   .withColumn("minimum_nights", col("minimum_nights").cast("integer"))
                   .withColumn("maximum_nights", col("maximum_nights").cast("integer")))
    
    df_calendar.write.mode("overwrite").parquet(path_silver_calendar)